# Teste 3 - Banco de Dados

### Etapa 1: Preparação dos Dados

Para preparação dos dados, realizamos antecipadamente o download dos dados dos últimos 2 anos das demonstrações contábeis como solicitado, e os zips de cada trimestre como foi encontrado, foram colocados dentro da pasta <code>demonstracoes_contabeis/zips</code>. Logo em seguida, usamos a biblioteca <code>zipfile</code> para manipular os zips e colocar todos os csvs juntos na pasta <code>demonstracoes_contabeis/csvs</code>.

In [1]:
import pandas as pd
import os
import zipfile

In [2]:
zip_dir = 'demonstracoes_contabeis/zips'
csv_dir = 'demonstracoes_contabeis/csvs'

In [3]:
for filename in os.listdir(zip_dir):
    with zipfile.ZipFile(f'{zip_dir}/{filename}', 'r') as zip_ref:    
        for csv_file in zip_ref.namelist():
            zip_ref.extract(csv_file, csv_dir)

Em seguida, realizamos a junção de todos os csvs em um único. Dando uma olhada nos arquivos csv separadamente, é possível observar diferenças nos tipos de armazenamento entre eles, por isso realizamos uma pequena manipulação usando <code>Pandas</code> para uma junção consistente dos dados.

Primeiro as datas foram padronizadas no padrão <code>Y-m-d</code> utilizado do SQL. Alguns descrições tiveram seus <i>encodes</i> ajustados e os valores dos saldos foram padronizados no modelo decimal com <code>.</code> também utilizado pelo SQL.

In [4]:
df_all = []
for filename in os.listdir(csv_dir):
    df = pd.read_csv(f'{csv_dir}/{filename}',delimiter=';',encoding='latin-1')
    
    df['DATA'] = pd.to_datetime(df['DATA'], infer_datetime_format=True)
    df['DATA'] = df['DATA'].dt.strftime('%Y-%m-%d')
    
    if 'VL_SALDO_INICIAL' in df:
        df['VL_SALDO_INICIAL'] = df['VL_SALDO_INICIAL'].apply(lambda x:float(x.replace(",",".")))
    df['VL_SALDO_FINAL'] = df['VL_SALDO_FINAL'].apply(lambda x:float(x.replace(",",".")))
    
    try:
        df['DESCRICAO'] = df['DESCRICAO'].apply(lambda x: x.encode("latin-1").decode("utf-8"))
    except:
        pass
    
    df_all.append(df)

Logo, foi possível realizarmos a junção desejada e salvamos tudo em um único csv:

In [7]:
result = pd.concat(df_all)
result.to_csv('demonstracoes_contabeis/all_data.csv', sep=";", index=False)

In [8]:
result.head(20)

,DATA,REG_ANS,CD_CONTA_CONTABIL,DESCRICAO,VL_SALDO_INICIAL,VL_SALDO_FINAL
0,2022-01-01,406805,46511,IMPOSTOS,0.0,2574.46
1,2022-01-01,406805,465119,Impostos,0.0,2574.46
2,2022-01-01,406805,46511901,Impostos,0.0,2574.46
3,2022-01-01,406805,465119012,Impostos Estaduais,0.0,2570.46
4,2022-01-01,406805,465119019,Impostos Municipais,0.0,4.00
5,2022-01-01,406805,4652,CONTRIBUIÇÕES,0.0,11107.19
6,2022-01-01,406805,46521,CONTRIBUIÇÕES,0.0,11107.19
7,2022-01-01,406805,465219,Contribuições,0.0,11107.19
8,2022-01-01,406805,46521901,Contribuições,0.0,11107.19
9,2022-01-01,406805,465219019,Outras Contribuições,0.0,11107.19


### Etapa 2: Manipulando o SQL

Feito o donwload do arquivo csv enviado para o Teste, exploramos ele um pouco para a criação do script de SQL a ser executado no Banco de Dados.

In [10]:
df = pd.read_csv("Relatorio_cadop(1) (2) (1).csv",skiprows=2,delimiter=';',encoding='latin-1')
df.head(5)

[Dados confidenciais]

In [11]:
df.columns

Index(['Registro ANS', 'CNPJ', 'Razão Social', 'Nome Fantasia', 'Modalidade',
       'Logradouro', 'Número', 'Complemento', 'Bairro', 'Cidade', 'UF', 'CEP',
       'DDD', 'Telefone', 'Fax', 'Endereço eletrônico', 'Representante',
       'Cargo Representante', 'Data Registro ANS'],
      dtype='object')

Logo, sabendo das colunas necessárias, criamos a <i>query</i> que irá criar a tabela de registros das operadoras nomerada de <code>operadoras_ativas</code>. Primeiro, é claro, criando nosso Banco de Dados e usando ele:

E em seguida, exploraremos também o csv com todas as demostrações contáveis para a criação da tabela que irá armazenar tais dados:

In [13]:
result.columns

Index(['DATA', 'REG_ANS', 'CD_CONTA_CONTABIL', 'DESCRICAO', 'VL_SALDO_INICIAL',
       'VL_SALDO_FINAL'],
      dtype='object')

Assim, temos o seguinte script para a criação da tabela relacionada a qual nomearemos de <code>demonstracoes_contabeis</code>:

E por fim, criaremos os scripts que carregará os dados dos nossos csvs para as nossas tabelas: